In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [2]:
data = pd.read_csv('../../data/telecom_churn.csv')

In [3]:
data.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [4]:
data.drop(['State', 'Voice mail plan'], axis=1, inplace=True)

In [5]:
data['International plan'] = data['International plan'].map({'Yes': 1, 'No': 0})

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Account length          3333 non-null   int64  
 1   Area code               3333 non-null   int64  
 2   International plan      3333 non-null   int64  
 3   Number vmail messages   3333 non-null   int64  
 4   Total day minutes       3333 non-null   float64
 5   Total day calls         3333 non-null   int64  
 6   Total day charge        3333 non-null   float64
 7   Total eve minutes       3333 non-null   float64
 8   Total eve calls         3333 non-null   int64  
 9   Total eve charge        3333 non-null   float64
 10  Total night minutes     3333 non-null   float64
 11  Total night calls       3333 non-null   int64  
 12  Total night charge      3333 non-null   float64
 13  Total intl minutes      3333 non-null   float64
 14  Total intl calls        3333 non-null   

In [7]:
y = data['Churn'].astype('int')

In [8]:
X = data.drop('Churn', axis=1)

In [9]:
X.shape, y.shape

((3333, 17), (3333,))

In [10]:
from sklearn.model_selection import train_test_split, cross_val_score

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.3, 
                                                      random_state=17)

In [12]:
X_train.shape, X_valid.shape

((2333, 17), (1000, 17))

In [13]:
first_tree = DecisionTreeClassifier(random_state=17)

In [14]:
cross_val_score(first_tree, X_train, y_train, cv=5)

array([0.9143469 , 0.91220557, 0.92077088, 0.90772532, 0.91416309])

In [15]:
np.mean(cross_val_score(first_tree, X_train, y_train, cv=5))

0.9138423504976518

In [16]:
from sklearn.neighbors import KNeighborsClassifier

In [17]:
first_knn = KNeighborsClassifier()

In [18]:
np.mean(cross_val_score(first_knn, X_train, y_train, cv=5))

0.8671274043984523

## настраиваем max_depth для дерева

In [19]:
from sklearn.model_selection import GridSearchCV

In [20]:
tree_params = {'max_depth': np.arange(1, 11), 'max_features':[.5, .7, 1]}

In [21]:
tree_grid = GridSearchCV(first_tree, tree_params, cv=5, n_jobs=-1)

In [22]:
%%time
tree_grid.fit(X_train, y_train);

Wall time: 2.61 s


GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=17,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 1

In [23]:
tree_grid.best_score_, tree_grid.best_params_

(0.9391366681677404, {'max_depth': 6, 'max_features': 0.7})

In [24]:
knn_params = {'n_neighbors': list(range(5, 30, 5)) + list(range(50, 100, 10))}

In [25]:
knn_grid = GridSearchCV(first_knn, knn_params, cv=5, n_jobs=-1)

In [26]:
%%time
knn_grid.fit(X_train, y_train);

Wall time: 406 ms


GridSearchCV(cv=5, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=-1,
             param_grid={'n_neighbors': [5, 10, 15, 20, 25, 50, 60, 70, 80,
                                         90]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [27]:
knn_grid.best_score_, knn_grid.best_params_

(0.8701289391697531, {'n_neighbors': 10})

In [28]:
tree_grid.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=6, max_features=0.7, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=17, splitter='best')

In [29]:
tree_valid_pred = tree_grid.predict(X_valid)

In [30]:
tree_grid.score(X_valid, y_valid)

0.936

In [31]:
from sklearn.metrics import accuracy_score

In [32]:
accuracy_score(y_valid, tree_valid_pred)

0.936

In [33]:
np.bincount(y_valid)

array([867, 133], dtype=int64)

In [34]:
from sklearn.metrics import confusion_matrix

In [35]:
confusion_matrix(y_valid, tree_valid_pred)

array([[858,   9],
       [ 55,  78]], dtype=int64)

In [36]:
from sklearn.tree import export_graphviz

In [37]:
export_graphviz(tree_grid.best_estimator_, out_file='telecom_tree.dot',
               feature_names=X.columns, filled=True,)

In [38]:
import pydotplus

In [39]:
graph = pydotplus.graph_from_dot_file('telecom_tree.dot')

In [40]:
import os
os.environ["PATH"] += os.pathsep + 'C:\Program Files (x86)\Graphviz2.38/bin/'
graph.write('telecom_tree.png')#,format="png")

True

In [41]:
os.environ["PATH"]

'D:\\anaconda3;D:\\anaconda3\\Library\\mingw-w64\\bin;D:\\anaconda3\\Library\\usr\\bin;D:\\anaconda3\\Library\\bin;D:\\anaconda3\\Scripts;C:\\Windows\\system32;C:\\Windows;C:\\Windows\\System32\\Wbem;C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\;C:\\Windows\\System32\\OpenSSH\\;C:\\Program Files\\Common Files\\Reallusion\\CamSuite;C:\\Program Files (x86)\\Common Files\\Reallusion\\CamSuite;C:\\Program Files\\Git\\cmd;D:\\Python 3.7.4\\Scripts\\;D:\\Python 3.7.4\\;C:\\Users\\Виталий\\AppData\\Local\\Microsoft\\WindowsApps;C:\\Program Files\\JetBrains\\PyCharm Community Edition 2020.1\\bin;;;C:\\Program Files (x86)\\Graphviz2.38/bin/'

<img src='telecom_tree.png'>

In [42]:
second_tree = DecisionTreeClassifier(max_depth=3).fit(X_train, y_train)
second_tree.score(X_valid, y_valid)

0.905

In [43]:
from io import StringIO
from ipywidgets import Image

dot_data = StringIO()
export_graphviz(decision_tree=second_tree,
               out_file=dot_data, filled=True, 
                feature_names=X.columns)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(value=graph.create_png())

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\xfd\x00\x00\x01\xf1\x08\x06\x00\x00\x00\x16Y\xc9…